import packages
------------------

Some packages are installed automatically if you use Anaconda. As pytorch is used here, you are expected to install that in your machine. 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import sys
sys.path.append('/content/gdrive/MyDrive/UTRECHT/utils')
sys.path.insert(0,'/content/gdrive/MyDrive/UTRECHT')

import utils

In [ ]:
from __future__ import print_function, division
import os, random, time, copy
from skimage import io, transform, morphology, feature
import numpy as np
import os.path as path
import scipy.io as sio
from scipy import misc
from scipy import ndimage, signal
import scipy
import pickle
import sys
import math
import matplotlib.pyplot as plt
import PIL.Image
from io import BytesIO
from skimage import data, img_as_float
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler 
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
import shutil

from utils.flow_functions import *

import warnings # ignore warnings
warnings.filterwarnings("ignore")
print(sys.version)
print(torch.__version__)

3.8.16 (default, Dec  7 2022, 01:12:13) 
[GCC 7.5.0]
1.13.0+cu116


Setup config parameters
 -----------------
 
 There are several things to setup, like which GPU to use, where to read images and save files, etc. Please read and understand this. By default, you should be able to run this script smoothly by changing nothing.

In [ ]:
### set path to image stacks, masks, and annotation files
# path_to_image1 = '/content/gdrive/MyDrive/UTRECHT/Detection/PAL1999/C6/C6_tiles_withAnnot'
# path_to_label1 = '/content/gdrive/MyDrive/UTRECHT/Detection/PAL1999/C6/C6_masks2'

path_to_image2 = '/content/gdrive/MyDrive/UTRECHT/Detection/PAL1999/C3/C3_tiles_withAnnot'
path_to_label2 = '/content/gdrive/MyDrive/UTRECHT/Detection/PAL1999/C3/C3_masks2'
# path_to_annotCombo = '/content/gdrive/MyDrive/UTRECHT/Detection/AnnotCombo9_noNPP_circleMask'

In [ ]:
#### set project name, save directory for the project
project_name_T1 = 'generalizedDet_finetuned_2022_11_28_T1' ## C3 sample 43 was the training set
project_name_T2 = 'generalizedDet_finetuned_2022_11_28_T2'
project_name_T3 = 'generalizedDet_finetuned_2022_11_28_T3'
# project_name = 'generalizedDet_finetuned_2022_11_28_T1T2T3'
project_name = 'generalizedDet_finetuned_2023_01_10_T1T2T3_forExp2' # equivalent to 'generalizedDet_finetuned_2022_11_28_T1T2T3' .plk file

exp_dir = '/content/gdrive/MyDrive/exp/final' # experiment directory, used for reading the init model
save_dir = os.path.join(exp_dir, project_name) 
if not os.path.exists(save_dir): 
    os.makedirs(save_dir)

In [ ]:
# see the lengths of the training and test datasets
projects = [project_name_T1,project_name_T2,project_name_T3]
train_det_list_full = []
test_det_list_full = []

for project in projects:
  with open(os.path.join(exp_dir,project,'dbinfo.plk'), 'rb') as handle:
      dbinfo = pickle.load(handle)
      for i in dbinfo['train_det_list']:
        train_det_list_full.append(i)
      for i in dbinfo['test_det_list']:
        test_det_list_full.append(i)
      # print(dbinfo['train_det_list'], dbinfo['test_det_list'])

print(len(train_det_list_full), len(test_det_list_full))

720 180


In [ ]:
# save all the info in a pickle file to be read in the next notebook

dbinfo_full = {}
dbinfo['train_det_list'] = train_det_list_full
dbinfo['test_det_list'] = test_det_list_full

with open(os.path.join(save_dir,'dbinfo.plk'), 'wb') as handle:
    pickle.dump(dbinfo, handle) # , protocol=pickle.HIGHEST_PROTOCOL



Leaving Blank
-----